In [1]:
import geopandas
import xarray
import rioxarray
import rasterio
from shapely.geometry import mapping
import glob
import os

In [63]:
shapefile = 'C:/Mayank/Projects/Project_Viatris/Data/Nashik_Bengaluru/Study_Area_1/Study_Area_1/Study_Area.shp'
sf = geopandas.read_file(shapefile)

In [68]:
# inp_dir = 'C:/Mayank/Projects/Project_Viatris/Data/BiasCorrection_Rainfall/Corrected/RCP26/'
inp_dir = 'C:/Mayank/Projects/Project_Viatris/Data/FromJuna/Tas/Tas/Regridded/Historical'

In [67]:
out_dir = 'C:/Mayank/Projects/Project_Viatris/Data/Output/tojuna/Banglore/Tas/'

In [69]:
file_nc = glob.glob(os.path.join(inp_dir,'*.nc'))
# print(file_nc)

for file in file_nc:
#     file_n = xarray.open_dataset(file)
    basename = os.path.basename(file)
    print(file)
    # Then apply the mask
#     masked_netcdf_file = nc_file.where(shape_mask == True, drop=True)
    # store again as netcdf or do what every you want with the masked array
    nc_file = xarray.open_dataset(file)
    nc_file.rio.write_crs('epsg:4326', inplace = True)
#     print(nc_file)
    dicts = nc_file.dims
#     newdict = {}
#     blklist= set(('bnds'))
#     for k in dicts:
#         if k not in blklist:
#             newdict[k]=dicts[k]
        
    if 'bnds' in dict(dicts):
        print('changing dims')
#         nc_file.rio.set_spatial_dims('lon','lat',inplace=True)
#         print(nc_file)
        nc_file2=nc_file.drop_vars('time_bnds', errors='raise')
        print(nc_file2)
        clipped_nc = nc_file2.rio.clip(sf.geometry.apply(mapping), sf.crs, all_touched = True)
        clipped_nc.rio.write_crs('epsg:4326', inplace=True)
        clipped_nc.to_netcdf(os.path.join(out_dir, basename))
#           nc_file.transpose('time','bnds','lon','lat')[['time','lon','lat','bnds']]
#     print("This is new dict", newdict)
#     nc_file.rename_dims(newdict)
# masked_netcdf_file = nc_file.where(shape_mask == True, drop = True)
#     print("new dimensions", nc_file.dims)
    else:
        clipped_nc = nc_file.rio.clip(sf.geometry.apply(mapping), sf.crs, all_touched = True)
        clipped_nc.rio.write_crs('epsg:4326', inplace=True)
        clipped_nc.to_netcdf(os.path.join(out_dir, basename))

C:/Mayank/Projects/Project_Viatris/Data/FromJuna/Tas/Tas/Regridded/Historical\CanESM_1990_2005_try2.nc
C:/Mayank/Projects/Project_Viatris/Data/FromJuna/Tas/Tas/Regridded/Historical\CCSM4_1990_2005_regridded.nc
changing dims
<xarray.Dataset>
Dimensions:      (time: 5840, lon: 31, lat: 31)
Coordinates:
  * time         (time) object 1990-01-01 12:00:00 ... 2005-12-31 12:00:00
  * lon          (lon) float64 67.5 68.5 69.5 70.5 71.5 ... 94.5 95.5 96.5 97.5
  * lat          (lat) float64 7.5 8.5 9.5 10.5 11.5 ... 34.5 35.5 36.5 37.5
    height       float64 ...
    spatial_ref  int32 0
Data variables:
    tas          (time, lat, lon) float32 ...
Attributes: (12/37)
    CDI:                          Climate Data Interface version 2.0.4 (https...
    Conventions:                  CF-1.4
    source:                       CCSM4
    institution:                  NCAR (National Center for Atmospheric Resea...
    history:                      Fri Nov 17 13:06:48 2023: cdo remapcon,tmea...
    in